In [8]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import librosa
import pandas as pd
import os
import string
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
import sklearn as sk
from sklearn.ensemble import RandomForestClassifier
from joblib import dump, load
from sklearn.metrics import accuracy_score
import featuretools as ft
import numpy as np
from itertools import chain
from emotion_classification_model import EmotionClassifier, feature_names
import pandas as pd

import sys,os
from midi2audio import FluidSynth

def accuracy_metrics(y_test, y_pred):
    happy_classified = 0
    sad_classified = 0
    thriller_classified = 0

    happy_total_count = 0
    sad_total_count = 0
    thriller_total_count = 0

    y_test_list = list(y_test)
    for i in range(len(y_test_list)):
        if y_test_list[i] == 0:
            happy_total_count += 1
        if y_test_list[i] == 1:
            sad_total_count += 1
        if y_test_list[i] == 2:
            thriller_total_count += 1

        if y_test_list[i] == 0 and y_pred[i] == 0:
            happy_classified += 1
        if y_test_list[i] == 1 and y_pred[i] == 1:
            sad_classified += 1
        if y_test_list[i] == 2 and y_pred[i] == 2:
            thriller_classified += 1
            
    happy_accuracy = happy_classified/happy_total_count
    sad_accuracy = sad_classified/sad_total_count
    thriller_accuracy = thriller_classified/thriller_total_count
    
    return happy_accuracy, sad_accuracy, thriller_accuracy


In [9]:
# Convert all midi to wav files

def midi_to_wav(path):
    files=os.listdir(path)
    for f in files:
        if(f.split('.')[-1]=='mid' or f.split('.')[-1]=='midi'):
            FluidSynth().midi_to_audio(path + "/" + f, path + "/" + f+'-output.wav')

# midi_to_wav("/Users/gokul/Downloads/MTD/predict_tunes/thriller")
# midi_to_wav("/Users/gokul/Downloads/MTD/predict_tunes/happy")
# midi_to_wav("/Users/gokul/Downloads/MTD/predict_tunes/sad")

# midi_to_wav("/Users/gokul/Downloads/MTD/predict_tunes/happyzip")

In [19]:
# Model for initial collection without the anger dataset
# Random test/train split

ec = EmotionClassifier()
happyzipdf = pd.DataFrame(columns=list(feature_names))
initial_collection_features = pd.read_pickle("../artefacts" +"/"+ 'features.csv')

train = initial_collection_features
train['indexcol'] = range(1, train.shape[0] + 1)
train = train[train["label"] != "anger"]
le = LabelEncoder()
train["label_encoded"] = le.fit_transform(train["label"])
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
# print(le_name_mapping)
label = train["label_encoded"]
train = train.drop(columns=["label", "label_encoded"])
def featurize(entry):
    return tuple(entry)
train["featurize"] = train.apply(featurize, axis=1)

result_model = ""
threshold_accuracy = 0.0
for i in range(25):
    x_train, x_test, y_train, y_test = sk.model_selection.train_test_split(train, label, test_size=0.3)

#     print(x_train.shape[0], x_train.shape[1])
#     print(x_test.shape[0], x_test.shape[1])
#     print(y_train.shape[0])
#     print(y_test.shape[0])

    train_list = list(x_train["featurize"])
    initial_collection_model = RandomForestClassifier().fit(train_list, y_train)
    y_pred = initial_collection_model.predict(list(x_test["featurize"]))
    score = accuracy_score(y_test, y_pred)
    if score > threshold_accuracy:
        result_model = initial_collection_model
        threshold_accuracy = score
        happy_accuracy, sad_accuracy, thriller_accuracy = accuracy_metrics(y_test, y_pred)
        
print("Overall validation accuracy", threshold_accuracy)
print()
print("Happy validation accuracy", happy_accuracy)
print("Sad validation accuracy",  sad_accuracy)
print("Thriller validation accuracy", thriller_accuracy)

x_pred = result_model.predict(list(x_train["featurize"]))
happy_accuracy, sad_accuracy, thriller_accuracy = accuracy_metrics(list(y_train), x_pred)
print()
print("Happy train accuracy", happy_accuracy)
print("Sad train accuracy",  sad_accuracy)
print("Thriller train accuracy", thriller_accuracy)

dump(result_model, "../models" + "/" + 'initial_collection_model.joblib')

Overall validation accuracy 0.9868421052631579

Happy validation accuracy 0.9565217391304348
Sad validation accuracy 1.0
Thriller validation accuracy 1.0

Happy train accuracy 0.9848484848484849
Sad train accuracy 1.0
Thriller train accuracy 1.0


['../models/initial_collection_model.joblib']

In [20]:
# Model for final midi collection without the anger/Various dataset
# Random test/train split

ec = EmotionClassifier()
happyzipdf = pd.DataFrame(columns=list(feature_names))
final_midi_features = pd.read_pickle("../artefacts" +"/"+ 'predict_features_mid.csv')

train = final_midi_features
train['indexcol'] = range(1, train.shape[0] + 1)
train = train[train["label"] != "anger"]
le = LabelEncoder()
train["label_encoded"] = le.fit_transform(train["label"])
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
# print(le_name_mapping)
label = train["label_encoded"]
train = train.drop(columns=["label", "label_encoded"])
def featurize(entry):
    return tuple(entry)
train["featurize"] = train.apply(featurize, axis=1)

result_model = ""
threshold_accuracy = 0.0
for i in range(25):
    x_train, x_test, y_train, y_test = sk.model_selection.train_test_split(train, label, test_size=0.3)

#     print(x_train.shape[0], x_train.shape[1])
#     print(x_test.shape[0], x_test.shape[1])
#     print(y_train.shape[0])
#     print(y_test.shape[0])

    train_list = list(x_train["featurize"])
    final_midi_model = RandomForestClassifier().fit(train_list, y_train)
    y_pred = final_midi_model.predict(list(x_test["featurize"]))
    score = accuracy_score(y_test, y_pred)
    if score > threshold_accuracy:
        result_model = final_midi_model
        threshold_accuracy = score
        
        happy_accuracy, sad_accuracy, thriller_accuracy = accuracy_metrics(y_test, y_pred)
        
print("Overall validation accuracy", threshold_accuracy)
print()
print("Happy validation accuracy", happy_accuracy)
print("Sad validation accuracy",  sad_accuracy)
print("Thriller validation accuracy", thriller_accuracy)

x_pred = result_model.predict(list(x_train["featurize"]))
happy_accuracy, sad_accuracy, thriller_accuracy = accuracy_metrics(list(y_train), x_pred)
print()
print("Happy train accuracy", happy_accuracy)
print("Sad train accuracy",  sad_accuracy)
print("Thriller train accuracy", thriller_accuracy)


dump(result_model, "../models" + "/" + 'final_midi_model.joblib')

Overall validation accuracy 0.9117647058823529

Happy validation accuracy 0.9444444444444444
Sad validation accuracy 1.0
Thriller validation accuracy 0.375

Happy train accuracy 0.9791666666666666
Sad train accuracy 1.0
Thriller train accuracy 0.8125


['../models/final_midi_model.joblib']

In [21]:
# Model for final midi collection without the anger/Various dataset
# Random test/train split 30/30/30

ec = EmotionClassifier()
happyzipdf = pd.DataFrame(columns=list(feature_names))
final_midi_303030_features = pd.read_pickle("../artefacts" +"/"+'predict_features_mid.csv')

df_unskew = final_midi_303030_features
df_unskew = df_unskew[df_unskew["label"] != "anger"]
df1 = (df_unskew[df_unskew['label'] == 'sad']).sample(frac=.24)
df2 = (df_unskew[df_unskew['label'] == 'happy']).sample(frac=.44)
df3 = df_unskew[df_unskew['label'] == 'thriller']
frames = [df1, df2, df3]
result = pd.concat(frames)

train = result
train['indexcol'] = range(1, train.shape[0] + 1)
train = train[train["label"] != "anger"]
le = LabelEncoder()
train["label_encoded"] = le.fit_transform(train["label"])
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
# print(le_name_mapping)
label = train["label_encoded"]
train = train.drop(columns=["label", "label_encoded"])
def featurize(entry):
    return tuple(entry)
train["featurize"] = train.apply(featurize, axis=1)

result_model = ""
threshold_accuracy = 0.0
for i in range(25):
    x_train, x_test, y_train, y_test = sk.model_selection.train_test_split(train, label, test_size=0.3)
#     print(x_train.shape[0], x_train.shape[1])
#     print(x_test.shape[0], x_test.shape[1])
#     print(y_train.shape[0])
#     print(y_test.shape[0])
    train_list = list(x_train["featurize"])
    final_midi_303030_model = RandomForestClassifier().fit(train_list, y_train)
    y_pred = final_midi_303030_model.predict(list(x_test["featurize"]))
    score = accuracy_score(y_test, y_pred)
    if score > threshold_accuracy:
        result_model = final_midi_303030_model
        threshold_accuracy = score
        
        happy_accuracy, sad_accuracy, thriller_accuracy = accuracy_metrics(y_test, y_pred)
        
print("Overall validation accuracy", threshold_accuracy)
print()
print("Happy accuracy", happy_accuracy)
print("Sad accuracy",  sad_accuracy)
print("Thriller accuracy", thriller_accuracy)

x_pred = result_model.predict(list(x_train["featurize"]))
happy_accuracy, sad_accuracy, thriller_accuracy = accuracy_metrics(list(y_train), x_pred)
print()
print("Happy train accuracy", happy_accuracy)
print("Sad train accuracy",  sad_accuracy)
print("Thriller train accuracy", thriller_accuracy)

dump(result_model, "../models" + "/" + 'final_midi_303030_model.joblib')

Overall validation accuracy 0.9629629629629629

Happy accuracy 1.0
Sad accuracy 0.8888888888888888
Thriller accuracy 1.0

Happy train accuracy 1.0
Sad train accuracy 0.9523809523809523
Thriller train accuracy 1.0


['../models/final_midi_303030_model.joblib']

In [22]:
# Model for final midi collection without the anger/Various dataset
# Micro-managed test/train split -> Same percentage of songs in all genres for training and testing

ec = EmotionClassifier()
happyzipdf = pd.DataFrame(columns=list(feature_names))
final_midi_random_equity_features = pd.read_pickle("../artefacts" +"/"+'predict_features_mid.csv')

df_unskew = final_midi_random_equity_features
df_unskew = df_unskew[df_unskew["label"] != "anger"]
train_sad, test_sad = sk.model_selection.train_test_split((df_unskew[df_unskew['label'] == 'sad']), test_size=0.2)
train_happy, test_happy = sk.model_selection.train_test_split((df_unskew[df_unskew['label'] == 'happy']), test_size=0.2)
train_thriller, test_thriller = sk.model_selection.train_test_split((df_unskew[df_unskew['label'] == 'thriller']), test_size=0.2)

frames = [train_sad, train_happy, train_thriller]
train = pd.concat(frames)

frames = [test_sad, test_happy, test_thriller]
test = pd.concat(frames)

train = result
train['indexcol'] = range(1, train.shape[0] + 1)
train = train[train["label"] != "anger"]
le = LabelEncoder()
train["label_encoded"] = le.fit_transform(train["label"])
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
# print(le_name_mapping)
label = train["label_encoded"]
train = train.drop(columns=["label", "label_encoded"])
def featurize(entry):
    return tuple(entry)
train["featurize"] = train.apply(featurize, axis=1)

result_model = ""
threshold_accuracy = 0.0
for i in range(25):
    x_train, x_test, y_train, y_test = sk.model_selection.train_test_split(train, label, test_size=0.3)
#     print(x_train.shape[0], x_train.shape[1])
#     print(x_test.shape[0], x_test.shape[1])
#     print(y_train.shape[0])
#     print(y_test.shape[0])
    train_list = list(x_train["featurize"])
    final_midi_random_equity_model = RandomForestClassifier().fit(train_list, y_train)
    y_pred = final_midi_random_equity_model.predict(list(x_test["featurize"]))
    score = accuracy_score(y_test, y_pred)
    if score > threshold_accuracy:
        result_model = final_midi_random_equity_model
        threshold_accuracy = score
        
        happy_accuracy, sad_accuracy, thriller_accuracy = accuracy_metrics(y_test, y_pred)
        
print("Overall validation accuracy", threshold_accuracy)
print()
print("Happy validation accuracy", happy_accuracy)
print("Sad validation accuracy",  sad_accuracy)
print("Thriller validation accuracy", thriller_accuracy)


x_pred = result_model.predict(list(x_train["featurize"]))
happy_accuracy, sad_accuracy, thriller_accuracy = accuracy_metrics(list(y_train), x_pred)
print()
print("Happy train accuracy", happy_accuracy)
print("Sad train accuracy",  sad_accuracy)
print("Thriller train accuracy", thriller_accuracy)

dump(result_model, "../models" + "/" + 'final_midi_random_equity_model.joblib')


Overall validation accuracy 0.9629629629629629

Happy validation accuracy 0.9230769230769231
Sad validation accuracy 1.0
Thriller validation accuracy 1.0

Happy train accuracy 0.9583333333333334
Sad train accuracy 1.0
Thriller train accuracy 1.0


['../models/final_midi_random_equity_model.joblib']

In [14]:
# Feature Extraction happyzipdf generated songs

import librosa
import pandas as pd
import os
import string
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
import sklearn as sk
from sklearn.ensemble import RandomForestClassifier
from joblib import dump, load
from sklearn.metrics import accuracy_score
import featuretools as ft
import numpy as np
from itertools import chain
from emotion_classification_model import EmotionClassifier, feature_names
import pandas as pd

import sys,os
from midi2audio import FluidSynth

ec = EmotionClassifier()

# happyzipdf = pd.DataFrame(columns=list(feature_names))
# happyzipdf = ec.add_features(happyzipdf, "/Users/gokul/Downloads/MTD/predict_tunes/happyzip", "happy", ".wav")
# happyzipdf.to_pickle('happyzipdf.csv')

In [25]:
# from emotion_classification_model import EmotionClassifier, feature_names
# from joblib import dump, load
# from sklearn.metrics import accuracy_score
# import os
# from mido import MidiFile
# from pydub import AudioSegment
# from sklearn.preprocessing import LabelEncoder
# import pandas as pd


# happyzipdf = pd.read_pickle("../artefacts" +"/"+ 'happyzipdf.csv')
# happyzipdf['indexcol'] = range(1, happyzipdf.shape[0] + 1)
# le = LabelEncoder()
# happyzipdf["label_encoded"] = le.fit_transform(happyzipdf["label"])
# le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
# print(le_name_mapping)
# label = happyzipdf["label_encoded"]
# happyzipdf = happyzipdf.drop(columns=["label", "label_encoded"])
# happyzipdf = happyzipdf.fillna(0)
# happyzipdf["featurize"] = happyzipdf.apply(ec.featurize, axis=1)

# model = load("../models" + "/" + "'initial_collection_model.joblib'")
# y_pred = model.predict(list(happyzipdf["featurize"]))
# print(accuracy_score(label, y_pred))

# model = load("../models" + "/" + 'final_midi_model.joblib')
# y_pred = model.predict(list(happyzipdf["featurize"]))
# print(accuracy_score(label, y_pred))

# model = load("../models" + "/" + 'final_midi_303030_model.joblib')
# y_pred = model.predict(list(happyzipdf["featurize"]))
# print(accuracy_score(label, y_pred))

# model = load("../models" + "/" + 'final_midi_random_equity_model.joblib')
# y_pred = model.predict(list(happyzipdf["featurize"]))
# print(accuracy_score(label, y_pred))

In [26]:
def midi_to_wav(path):
    files=os.listdir(path)
    for f in files:
        if(f.split('.')[-1]=='mid' or f.split('.')[-1]=='midi'):
            FluidSynth().midi_to_audio(path + "/" + f, path + "/" + f+'-output.wav')

midi_to_wav("/Users/gokul/Downloads/MTD/test")

In [27]:
ec = EmotionClassifier()
test_df = pd.DataFrame(columns=list(feature_names))

In [28]:
test_df

,a,b,c,d,e,f,g,h,i,j,...,at,au,av,aw,ax,ay,az,ba,bb,label
